# 필요한 패키지 설치

In [2]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn
from torch_geometric.nn import GCNConv

from torch_geometric.datasets import Planetoid

# Data 불러오기

In [2]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [3]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [4]:
num_node_features = dataset.num_node_features

In [5]:
num_classes = dataset.num_classes

# 모델 만들기

In [6]:
class GCN(nn.Module):
    
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x)
        x = self.conv2(x, edge_index)
        
        return F.softmax(x, dim=1)
        

# Learning

In [7]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [8]:
model = GCN()
model.to(device)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [10]:
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
epochs = 200

In [11]:
for epoch in range(epochs):
    model.train()
    # forward
    pred = model(data)
    
    # loss
    loss = F.cross_entropy(pred[data.train_mask], data.y[data.train_mask])
    
    # backward
    loss.backward()
    
    # update
    optimizer.step()
    
    train_loss = loss.item()
    
    model.eval()
    pred = model(data).max(dim=1).indices
    correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    
    if (epoch+1) % 10 == 0:
        print('epoch : {}, train_loss : {}, acc : {}'.format(epoch+1, train_loss, acc))

epoch : 10, train_loss : 1.6770507097244263, acc : 0.422
epoch : 20, train_loss : 1.3932958841323853, acc : 0.603
epoch : 30, train_loss : 1.256840467453003, acc : 0.748
epoch : 40, train_loss : 1.3130792379379272, acc : 0.661
epoch : 50, train_loss : 1.2785279750823975, acc : 0.618
epoch : 60, train_loss : 1.2168068885803223, acc : 0.651
epoch : 70, train_loss : 1.195411205291748, acc : 0.673
epoch : 80, train_loss : 1.1853688955307007, acc : 0.654
epoch : 90, train_loss : 1.202755331993103, acc : 0.653
epoch : 100, train_loss : 1.2147891521453857, acc : 0.669
epoch : 110, train_loss : 1.205511450767517, acc : 0.664
epoch : 120, train_loss : 1.2024109363555908, acc : 0.657
epoch : 130, train_loss : 1.1868870258331299, acc : 0.674
epoch : 140, train_loss : 1.1818103790283203, acc : 0.673
epoch : 150, train_loss : 1.1914353370666504, acc : 0.699
epoch : 160, train_loss : 1.1938915252685547, acc : 0.671
epoch : 170, train_loss : 1.1797077655792236, acc : 0.666
epoch : 180, train_loss : 1

In [15]:
data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]], device='cuda:0')

In [21]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
maybe_num_nodes(data.edge_index, None)

2708

In [26]:
torch.ones(100).size()

torch.Size([100])

In [34]:
data.edge_index.size(1)

10556

In [38]:
edge_weight = torch.ones((data.edge_index.size(1), ),device=data.edge_index.device)

In [39]:
from torch_geometric.utils import add_remaining_self_loops
data.edge_index, tmp_edge_weight = add_remaining_self_loops(
                data.edge_index, edge_weight, 2., data.num_nodes)

In [53]:
row, col = data.edge_index[0], data.edge_index[1]

In [3]:
from torch_geometric.datasets import Planetoid

In [6]:
dataset='Cora'
path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'dataset')
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

NameError: name '__file__' is not defined

In [11]:
import os.path as osp
import torch_geometric.transforms as T

In [12]:
path = '../data/dataset'

In [13]:
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())

Processing...
Done!


In [14]:
data = dataset[0]

In [16]:
dir(data)

['__apply__',
 '__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply',
 'clone',
 'coalesce',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'debug',
 'edge_attr',
 'edge_index',
 'face',
 'from_dict',
 'is_coalesced',
 'is_directed',
 'is_undirected',
 'keys',
 'normal',
 'num_edge_features',
 'num_edges',
 'num_faces',
 'num_features',
 'num_node_features',
 'num_nodes',
 'pos',
 'test_mask',
 'to',
 'to_dict',
 'to_namedtuple',
 'train_mask',
 'val_mask',
 'x',
 'y']

In [20]:
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [21]:
data.test_mask

tensor([False, False, False,  ...,  True,  True,  True])